In [2]:
import math

def information_gain(x, y, impurity_measure="entropy"):
    mean = 0
    count = 0
    for i in range(len(x)):
        mean += x[i]
        count +=1
    mean /= count
    leq_mean = []
    g_mean = []
    count_leq_mean = 0
    for i in range(len(x)):
        if(x[i] <= mean):
            count_leq_mean +=1
            leq_mean.append(y[i])
        else:
            g_mean.append(y[i])
    prob_leq_mean = count_leq_mean / count
    if(impurity_measure=="entropy"):
        impurity = entropy(y)
        conditional_impurity = prob_leq_mean * entropy(leq_mean)+ (1-prob_leq_mean) * entropy(g_mean)
    elif(impurity_measure=="gini"):
        impurity = gini(y)
        conditional_impurity = prob_leq_mean * gini(leq_mean)+ (1-prob_leq_mean) * gini(g_mean) 
    else:
        print("Not known impurity measure")
    return impurity - conditional_impurity

def probabiliy_list(values):
    count_values = len(values)
    value_list = []
    prob_list = []
    different_val = 0
    for i in range(count_values):
        if (values[i] not in value_list): 
            value_list.append(values[i])
            prob_list.append(1)
            different_val +=1
        else:
            for j in range(different_val):
                if(values[i]==value_list[j]): prob_list[j]+=1
    for k in range(different_val):
        prob_list[k] = float(prob_list[k]) / count_values
    return prob_list 

def entropy(values):
    prob_list = probabiliy_list(values)
    different_values = len(prob_list)
    sum = 0
    for i in range(different_values):
        sum -= math.log(prob_list[i], 2) * prob_list[i]
    return sum

def gini(values):
    prob_list = probabiliy_list(values)
    different_values = len(prob_list)
    sum = 0
    for i in range(different_values):
        sum += prob_list[i]*(1-prob_list[i])
    return sum



In [5]:
list = [1,1,2,2]
print(entropy(list))
print(gini(list))
list2 = [1,1,1,2]
print(entropy(list2))
print(gini(list2))


1.0
0.5
0.8112781244591328
0.375


In [3]:
import numpy as np

def delete_column(x,index):
    np.delete(x,index,0)


#class has variables: data, children
#if leaf: data is label otherwise data is featureindex
#children is a list with elements of form (value, node)
#Nodes save sorted listed of values --> check < on everyone except last
def predict(x):
    count_values = len(children)
    for i in range(count_values):
        if (not children):                          #empty children list --> leave
            return data
        elif(i == count_values-1):                  #Last value in list --> x > elements in this tree
            x_update = delete_column(x, data)       #not leafe --> data=feature_index
            (children[i])[1].predict(x_update)      #Remark: already used one feature!
        elif(x[data] < (children[i])[0]):           #search child with right value
            x_update = delete_column(x, data)
            (children[i])[1].predict(x_update)      #use predict on child recursivly

In [6]:
#Condition: Memorized majority label when building tree! --> majority_label

def majority_label_acc(majority_label, y_prun):
    x = 0
    for i in range(len(y_prun)):
        if(y_prun[i]==majority_label):
            x += 1
    return float(x) / len(y_prun)

def acc(X, Y):
    x = 0
    for i in range(len(Y)):
        if(Y[i]==predict(X[i])):#predict(X[i]) == i-th data-point in X
            x += 1
    return float(x) / len(Y)

def delete_feature_from_matrix(X, Feature_index):
    np.delete(X, Feature_index, 1)#Feature is Axis 1
    
def get_majority_label(Y):
    count_values = len(Y)
    label_list = []
    count_list = []
    different_val = 0
    for i in range(count_values):
        if (Y[i] not in label_list): 
            label_list.append(Y[i])
            count_list.append(1)
            different_val +=1
        else:
            for j in range(different_val):
                if(Y[i]==label_list[j]): count_list[j]+=1
    

#this method has to be defined within the tree class
def pruning(X_prun, y_prun): #starts with root + recursion
    if (children): #the subtree is not a leaf
        if (acc(X_prun, y_prun) < majority_label_acc(y_prun)): #case: switch subtree with majority_label
            data = majority_label
            children = []
        else: 
            for i in range(len(children)):
                X_prun_update = delete_feature_from_matrix(X_train, data)#not leaf --> data = feature_index
                (children[i])[1].pruning(X_prun_update, y_prun) #call recursion on each child



In [ ]:
import matplotlib.pyplot as plt
from sklearn import model_selection


X,Y = ...
seed = 666
X_train, X_val_test, Y_train, Y_val_test = model_selection.train_test_split(X, Y, test_size= 0.3, shuffle=True,                                                                                     random_state = seed)
seed = 221
X_val, X_test, Y_val, Y_test = model_selection.train_test_split(X, Y, test_size= 0.5, shuffle=True,                                                                                     random_state = seed)
#We now have ratio training-, validation-, testdata: 0.7, 0.15, 0.15

#Train decisiontree using entropy + pruning
tree_ent_prun = tree.initialise() #no idea how implemented
tree_ent_prun.learn(X_train, Y_train, impurity_measure='entropy', pruning=True)
val_acc_ent_prun = tree_ent_prun.acc(X_val, Y_val)
train_acc_ent_prun = tree_ent_prun.acc(X_train, Y_val)

#Train decisiontree using gini + pruning
tree_gini_prun = tree.initialise()
tree_gini_prun.learn(X_train, Y_train, impurity_measure='gini', pruning=True)
val_acc_gini_prun = tree_gini_prun.acc(X_val, Y_val)
train_acc_gini_prun = tree_gini_prun.acc(X_train, Y_val)

#Train decisiontree using entropy and not pruning
tree_ent_notprun = tree.initialise()
tree_ent_notprun.learn(X_train, Y_train, impurity_measure='entropy', pruning=False)
val_acc_ent_notprun = tree_ent_notprun.acc(X_val, Y_val)
train_acc_ent_notprun = tree_ent_notprun.acc(X_train, Y_train)

#Train decisiontree using gini and not pruning
tree_gini_notprun = tree.initialise()
tree_gini_notprun.learn(X_train, Y_train, impurity_measure='gini', pruning=False)
val_acc_gini_notprun = tree_gini_notprun.acc(X_val, Y_val)
train_acc_gini_notprun = tree_gini_notprun.acc(X_train, Y_train)

#plot accuracies
settings_list = [ent_prun, gini_prun, ent_notprun, gini_notprun]
train_accuracies = [train_acc_ent_prun, train_acc_gini_prun, train_acc_ent_notprun, train_acc_gini_notprun]
val_accuracies = [val_acc_ent_prun, val_acc_gini_prun, val_acc_ent_notprun, val_acc_gini_notprun]
plt.plot(settings_list, train_accuracies)
plt.plot(settings_list, val_accuracies)
plt.ylim(0, 1)
plt.xlabel('Settings')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='best')
plt.title("Decision-tree accuracies with different settings")

plt.show()
